In [1]:
from google.colab import auth
auth.authenticate_service_account()

Successfully saved credentials for pipelineauth@weatherlink-404323.iam.gserviceaccount.com


In [2]:
from google.cloud import bigquery
project_name='weatherlink-404323'
client = bigquery.Client(project=project_name)

In [3]:
dataset_name = 'weatherlink_master'

In [202]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pickle

In [187]:
sql_query = """

select
a.*,
b.count,

FROM
    `weatherlink_master.census_accident_master_filtered` a
JOIN
    (
SELECT
  count(CONCAT(geoid, week_of_crash, day_of_week_name)) as count,  geoid, week_of_crash, day_of_week_name
FROM
  `weatherlink_master.census_accident_master_filtered`
GROUP BY
  geoid, week_of_crash, day_of_week_name

order by count desc

) b
ON
    a.geoid = b.geoid AND a.week_of_crash = b.week_of_crash and a.day_of_week_name = b.day_of_week_name

"""

df = client.query(sql_query).to_dataframe()
print(df)

        geoid timestamp_of_crash  year  month_of_crash  week_of_crash  \
0       12033         2016-02-28  2016               2              9   
1       12033         2016-05-15  2016               5             20   
2       12033         2016-03-31  2016               3             13   
3       12033         2016-05-06  2016               5             18   
4       12033         2016-09-03  2016               9             35   
...       ...                ...   ...             ...            ...   
115647  55039         2019-12-05  2019              12             48   
115648  55039         2019-10-14  2019              10             41   
115649  55039         2019-09-01  2019               9             35   
115650  55039         2019-01-13  2019               1              2   
115651  55039         2019-06-06  2019               6             22   

       day_of_week_name  hour_of_crash atmospheric_cond_name  \
0                Sunday             17                 Clea

In [188]:
df

,geoid,timestamp_of_crash,year,month_of_crash,week_of_crash,day_of_week_name,hour_of_crash,atmospheric_cond_name,manner_of_collision_name,number_of_fatalities,light_condition_name,no_car_ratio,total_pop,male_pop_ratio,median_rent,median_age,income_per_capita,count
0,12033,2016-02-28,2016,2,9,Sunday,17,Clear,Not a Collision with Motor Vehicle In-Transport,1,Dusk,7,315187,50,720,37,24607,1
1,12033,2016-05-15,2016,5,20,Sunday,1,Clear,Not a Collision with Motor Vehicle In-Transport,1,Dark - Not Lighted,7,315187,50,720,37,24607,1
2,12033,2016-03-31,2016,3,13,Thursday,15,Cloudy,Angle,1,Daylight,7,315187,50,720,37,24607,4
3,12033,2016-05-06,2016,5,18,Friday,4,Clear,Sideswipe - Opposite Direction,1,Dark - Lighted,7,315187,50,720,37,24607,1
4,12033,2016-09-03,2016,9,35,Saturday,22,Cloudy,Not a Collision with Motor Vehicle In-Transport,1,Dark - Not Lighted,7,315187,50,720,37,24607,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115647,55039,2019-12-05,2019,12,48,Thursday,1,Clear,The First Harmful Event was Not a Collision wi...,2,Dark - Not Lighted,20,103403,48,647,42,33847,1
115648,55039,2019-10-14,2019,10,41,Monday,15,Clear,Angle,1,Daylight,20,103403,48,647,42,33847,1
115649,55039,2019-09-01,2019,9,35,Sunday,2,Clear,The First Harmful Event was Not a Collision wi...,1,Dark - Lighted,20,103403,48,647,42,33847,1
115650,55039,2019-01-13,2019,1,2,Sunday,17,Clear,The First Harmful Event was Not a Collision wi...,1,Dark - Not Lighted,20,103403,48,647,42,33847,1


In [189]:
le = LabelEncoder()

In [190]:
df["day_of_week_name_le"] = le.fit_transform(df["day_of_week_name"])
df["light_condition_name_le"] = le.fit_transform(df["light_condition_name"])
df["atmospheric_cond_name_le"] = le.fit_transform(df["atmospheric_cond_name"])

In [191]:
X = df[['week_of_crash', 'day_of_week_name_le', 'light_condition_name_le', 'atmospheric_cond_name_le', 'no_car_ratio', 'male_pop_ratio', 'median_age', 'median_rent', 'income_per_capita' ]]
Y = df['count']

In [192]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2, random_state = 100)

In [193]:
dt = DecisionTreeRegressor()
#random_forest = RandomForestRegressor(max_depth=10)

In [194]:
dt.fit(X_train, y_train)
#random_forest.fit(X_train, y_train)

DecisionTreeRegressor()

In [195]:
dt.score(X_test, y_test)

0.7412455631043895

In [196]:
#random_forest.score(X_test, y_test)

In [197]:
#random_forest.feature_importances_

In [198]:
dt.feature_importances_

array([0.16849553, 0.07209046, 0.01442309, 0.00970654, 0.18265434,
       0.02689407, 0.12191629, 0.22609235, 0.17772734])

In [203]:
pickle.dump(dt, open("deicision_tree_0_74.sav", 'wb'))